In [2]:
import os
from tqdm import tqdm
from time import time
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

PATH_TO_DATA = ('D:/Anaconda/PyCharm1/')

!head -1 stackoverflow.10kk.tsv

%%time
!wc -l stackoverflow.10kk.tsv

SyntaxError: invalid syntax (<ipython-input-2-83f48a3a6cd5>, line 12)

In [3]:
!python preprocess_sof.py stackoverflow.10kk.tsv stackoverflow.vw
%%time
!wc -l stackoverflow.vw
!gzip stackoverflow.10kk.tsv

SyntaxError: invalid syntax (<ipython-input-3-2ca1a981e9b1>, line 2)

In [4]:
# делим файл на 3 части по 1.463.018 строк
# -a позволяет контролировать длину суффикса(в данном случае длина первого суффикса = 1)
# -d позволяет использовать числовой суффикс
!split -l 1463018 -a 1 - d stackoverflow.vw
# на выходе получим: x0, x1, x2. P.S. переименуем их ручками.


# векторы ответов
!cut -c 1 stackoverflow_valid.vw > stackoverflow_valid_labels.txt
!cut -c 1 stackoverflow_test.vw > stackoverflow_test_labels.txt

#ЭТО НЕ НУЖНО, ОТВЕТЫ НЕЛЬЗЯ УДАЛЯТЬ, но код просто так на будущее
# удаляем ответы из самих файлов(первые 4 строки), оставляя только текст
!sed -r 's/.{4}//' stackoverflow_train.vw

usage: split [-a sufflen] [-b byte_count] [-l line_count] [-p pattern]
             [file [prefix]]
cut: stackoverflow_valid.vw: No such file or directory
cut: stackoverflow_test.vw: No such file or directory
sed: illegal option -- r
usage: sed script [-Ealn] [-i extension] [file ...]
       sed [-Ealn] [-i extension] [-e script] ... [-f script_file] ... [file ...]


In [5]:
-d 'D:/Anaconda/PyCharm1/stackoverflow_train.vw' # путь к данным
-f -'D:/Anaconda/PyCharm1/vw_model_Xngram_Xpasses.vw'# путь к модели
-b 28 # бит для хэширования
--random_seed 18 # сид
--ngram X # Х = 1, 2, 3
--passes X # X = 1, 3, 5
-c # использование кэша(для passes= 3, 5)

./vw -t -i 'D:/Anaconda/PyCharm1/vw_model_Xngram_Xpasses.vw' # обученная модель
-d 'D:/Anaconda/PyCharm1/stackoverflow_valid.vw' # какую модель хотим проверить
-p 'D:/Anaconda/PyCharm1/vw_model_1ngram_1passes_pred.csv' # куда хотим записать предсказания
--random_seed 18

%%time
# Доля правильных ответов на отложенной выборке:
accuracy_list = []
for i in range(1, 4):
    for j in range(1, 7, 2):
        vw_pred = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_model_{}ngram_{}passes_pred.csv'.format(i, j)))
        valid_labels = np.loadtxt(os.path.join(PATH_TO_DATA, 'stackoverflow_valid_labels.txt'))
        score = accuracy_score(valid_labels, vw_pred)
        accuracy_list.append(score)
        print('Ngram =', i, 'Passes =', j, 'Accuracy =', score)
print('Best accuracy =', max(accuracy_list))

SyntaxError: invalid syntax (<ipython-input-5-edc4528effa4>, line 1)

In [6]:
./vw -t -i 'D:/Anaconda/PyCharm1/vw_model_2ngram_1passes.vw' # обученная модель
-d 'D:/Anaconda/PyCharm1/stackoverflow_test.vw' # какую модель хотим проверить
-p 'D:/Anaconda/PyCharm1/vw_model_2ngram_1passes_pred_for_test.csv' # куда хотим записать предсказания
--random_seed 18

vw_test = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_model_2ngram_1passes_pred_for_test.csv'))
test_labels = np.loadtxt(os.path.join(PATH_TO_DATA, 'stackoverflow_test_labels.txt'))
best_accuracy_1 = accuracy_score(test_labels, vw_test)
print(best_accuracy_1)

print('Difference =', round((best_accuracy_1 - max(accuracy_list)) * 100, 3))

SyntaxError: invalid syntax (<ipython-input-6-8e5a412a73e7>, line 1)

In [7]:
# соеденим два файла
!cd 'D:/Anaconda/PyCharm1/'
!cat stackoverflow_train.vw stackoverflow_valid.vw > stackoverflow_t_v.vw
# обучим модель на данных вдвое больше
./vw --oaa 10 -d 'D:/Anaconda/PyCharm1/stackoverflow_t_v.vw'
-f -'D:/Anaconda/PyCharm1/vw_best_model.vw'
-b 28 --random_seed 18 --ngram 2 --passes 1
# сделаем прогноз
./vw -t -i 'D:/Anaconda/PyCharm1/vw_best_model.vw' # обученная модель
-d 'D:/Anaconda/PyCharm1/stackoverflow_test.vw' # какую модель хотим проверить
-p 'D:/Anaconda/PyCharm1/vw_best_model_pred_for_test.csv' # куда хотим записать предсказания
--random_seed 18
vw_test = np.loadtxt(os.path.join(PATH_TO_DATA, 'vw_best_model_pred_for_test.csv'))
test_labels = np.loadtxt(os.path.join(PATH_TO_DATA, 'stackoverflow_test_labels.txt'))
best_accuracy_2 = accuracy_score(test_labels, vw_test)
print(best_accuracy_2)
print('Difference =', round((best_accuracy_2 - best_accuracy_1) * 100, 3))

SyntaxError: invalid syntax (<ipython-input-7-adce9d8017be>, line 5)